In [1]:
import numpy as np


# This is now Starcraft 0

We're in in the world of Braxis, home of the Protoss Empire. Full of resources (Mineral and Gas). With $w_0$ starting workers and a single mineral deposit. We've enjoyed peace time for as long as we've been us. But not for much longer. The dominion forces announced an incoming attack in 10 minutes. But they are so dumb that they told us their army composition. We finally will have to work our asses off and build the biggest army of the firstborn's history capable of countering theirs.

###### Hey you. You're finally awake...

You must help us instruct our $w_0$ starting units to gather resources to build the strongest army against theirs that is possible in 10 minutes.

The game works like this:

#### You're able to command the workers in real time

The workers have 4 actions:

- Do nothing

- Gather resources
- Return resources (1 second)
- Build structures

There are only 2 resources in the world of Braxis: Mineral (Blue crystals) and Gas (Green gas). 

you start with 100 minerals and 0 gas

It takes 1 second to gather 5 minerals per worker   
It takes 2 seconds to gather 5 gas

In [2]:
Starting_minerals = 100
Starting_gas      = 0

Their distribuition is given by the following:

There are 3 minerals spots with 1000 capacity each;
There are 2 gas geysers with 500 capacity each;

In [3]:
Starting_available_minerals    = np.array([1000,1000,1000])
Starting_available_gas_geysers = np.array([500,500])

There are only 3 kinds of structures in the game:

- Resource Deposits - Costs 100 minerals (gives 30 supply, holds 200 minerals and 200 gas)
- Barracks - costs 200 minerals            
- Nexus    - costs 1000 minerals (gives 10 supply, holds 100 minerals)

The game starts with 1 nexus, 0 barracks, and  0 RDs

RDs allows you to build more units and store more resources

Barracks are your army builder, they can train:
 - Zealots  - costs 150 minerals, 2 supply
 - Stalkers - costs 150 minerals, 50 gas, 3 supply

In [4]:
army_weight = np.array([2,3])

Nexi allows you to train workers (50 minerals and 1 supply)

We'll define for each worker one possible state, that will define its actions:
 - 0 = nothing
 - 1x = gathering (11 minerals, 12 gas)
 - 2x = building  (21 RDs, 22 Barrakcs, 23 Nexus)

Therefore, we have at the starting point of our empire with a single nexus and worker:


In [5]:
# Time will be measured in seconds
time     = 0        
time_lim = 600

# Starting Protoss Empire (time = 0)

In [6]:
# Structures
nexus    = 1
barracks = 0
RDs      = 0
structures = np.array([nexus, barracks, RDs])

In [7]:
# Units
workers        = np.array([11])
army_comp      = np.array([0,0])

units   = [workers, army_comp]


In [8]:
# Resources


army_weight = np.array([2,3])

def total_supply(units, weight):
    workers, army_comp = units
    return len(workers) + army_comp@weight

def maximum_supply(structures):
    return np.array([10,0,30])@structures

available_minerals = Starting_available_minerals
available_gas      = Starting_available_gas_geysers
current_minerals   = Starting_minerals
current_gas        = Starting_gas

resources = [current_minerals, available_minerals, current_gas, available_gas]

In [9]:
# Production
nexi_production     = []  # booleans
barracks_production = []  # tuple of booleans  -> (0,x) = nothing; (1,0) -> zealot; (1,1) -> stalker

#### Right now, we have the following optimization problem:

$$ \max \sum \mathbb{E}[\Phi (army(t)[i])| Enemy]$$
$$ \text{ s.t.  } t \leq 600 $$ 


In [10]:
## Phi é uma média ou probabilidade do exército vencer o exército inimigo

###### We can view it as a multi-stage optimization problem (with 600 stages), where whenever we start producing a unit in any structure, it  gets 'deactivated' for a certain period and then comes back with a new unit 

Now, the hardest part of the game: The Production.
Or, in other words, WHAT DO I DO WITH MY MONEY

1. We can only produce a unit if we have its required structure

2. We can only be producing a single unit per structure 

3. I think thats all

It takes 5 seconds to build a worker  
10 seconds to build a zealot  
12 seconds to build a stalker

# MAS ANTES DISSO TUDO AI BORA SIMPLIFICAR UM POUCO 

## TEM ESSA DE RD NÃO
## SÓ TEM 1 TIPO DE MINERIO
## SÓ TEM 1 UNIDADE DA BARRACA
#### Time_lim = 20 segundos

In [11]:
# Stages will be measured in turns
turn     = 0        
max_turn = 20

In [12]:
# Structures

nexus    = 1
barracks = 0

structures = np.array([nexus, barracks])

In [13]:
# Units
workers        = [[11,2]]
army_comp      = np.array([0])
units          = [workers, army_comp]

In [14]:
# Resources


army_weight = np.array([2])

def total_supply(units, weight):
    workers, army_comp = units
    return len(workers) + army_comp@weight

def maximum_supply(structures):
    return np.array([10,0])@structures

Starting_available_minerals = 250
#Starting_available_gas_geysers = 0
Starting_minerals = 50
#Starting_gas = 0

available_minerals = Starting_available_minerals
#available_gas      = Starting_available_gas_geysers
current_minerals   = Starting_minerals
#current_gas        = Starting_gas

resources = [current_minerals, available_minerals]

In [15]:
# Production
nexi_production     = []  # tuple of boolean + int -> (0,0), (1,t)
barracks_production = []  # tuple of tuple of booleans + int  -> (0,x),0 = nothing; (1,0),t -> zealot; (1,1),t -> stalker
production = [nexi_production,barracks_production]

In [16]:
def resources_tick(resources, units, structures, production):
    
    workers, army = units
    c_m, a_m      = resources
    new_workers   = []
    n_p, b_p      = production
    
    for probe in workers:
        action, time = probe
        if action:
            time -= 1
            if time == 0:
                if action == 11: # returned minerals
                    c_m, a_m = c_m +5, a_m-5
                    print(" You gathered 5 minerals!")
                    if a_m == 0:
                        new_workers.append([0,0])
                    else: 
                        new_workers.append([11,2])
                if action == 22: # built barrakcs
                    print(" You just build a new barrack!")
                    structures += np.array(0,1)
                    new_workers.append([0,0])
                    n_p.append([0,0])
                if action == 23: # built nexus
                    print("You just build a new nexus!!")
                    structures += np.array(1,0)
                    new_workers.append([0,0])
                    b_p.append([[0,0],0])
            else:
                new_workers.append([action,time])
        else:
            new_workers.append([0,0])
    
    new_resorces = [c_m, a_m]
    new_units    = [new_workers, army]
    
    return new_resorces, new_units, structures, production      

In [17]:
def production_tick(production, units):
    
    n_p, b_p    = production
    wrkrs, army = units
    new_production = [[],[]]
    for n in n_p:
        if n[0]:
            t1 = n[1]
            t1 -= 1
            if t1 == 0:
                print(" You just trained a new probe! ")
                wrkrs += [11]
                new_production[0].apppend([0,0])
            else:
                new_production[0].append([1,t1])
        else:
            new_production[0].append([0,0])
            
    for b in b_p:
        u,t2 = b
        if u[0]:
            t2 -= 1
            if t2 == 0:
                #if u[1] == 0:
                print("You just trained a zealot!")
                army[0] += 1
                new_production[1].append([[0,0],0])
#                 else:
#                     print("You just trained a stalker!")
#                     army[1] += 1

            else:
                new_production[1].append([u,t2])
        else:
            new_production[1].append([[0,0],0])
            
    new_units = [wrkrs, np.array(army)]
    
    return new_production, new_units

In [18]:
def give_new_instructions(instructions, units, resources, production, current_supply, max_supply):
    workers, army = units
    c_m, a_m      = resources
    n_p, b_p      = production
    
    for probe in workers:
        if probe[0] == 0:
            pass
    return None

# Aqui temos uma pequena simulação :)

In [19]:
while turn <= max_turn:
    
    turn += 1
    
    resources, units, structures, production    = resources_tick(resources, units, structures, production)   # workers tick
    production, units               = production_tick(production,units) # buildings tick
    current_supply, max_supply      = total_supply(units, army_weight), maximum_supply(structures)
    

 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!
 You gathered 5 minerals!


In [20]:
resources, units, structures, production

([100, 200], [[[11, 1]], array([0])], array([1, 0]), [[], []])

# Agora vamos fazer coisas muito punk:

- Uma relaxação contínua do problema ver no que dá
- Definir um jeito de transformar instruções em numeros e resolver com genético

## Depois eu complicarei o problema colocando estocasticiade (ataques bárbaros aleatórios/ tempo de vida das unidades ou coisas assim)

#### ver no que dá